In [1]:
import pandas as pd
import numpy as np
import sys, zipfile, os, time
import matplotlib.pyplot as plt
import glob
import datetime
import urllib3
import cdsapi
import xarray
import shutil
import warnings
import pickle
import h5py
import netCDF4
warnings.filterwarnings('ignore')

sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua3 as eua

In [2]:
def get_adj(station, tarvar):
    import datetime
    if tarvar == 'directionbias':
        cdsvar = 'wind_direction'
    elif tarvar == 'uwindbias':
        cdsvar = 'u_component_of_wind'
    elif tarvar == 'uwindbias':
        cdsvar = 'u_component_of_wind'
    else:
        print('not an available wind variable')
        return 0, 0, 0, 0, 0
    """Compare created adjustment to downloaded adjustment
    """
    date0 = datetime.datetime.strptime('1900-01-01', "%Y-%m-%d")

    file = glob.glob('./Wind_adjustment/*/*winddir*'+station+'*')[0]
    data = eua.CDMDataset(file)#.to_dataframe()
    display(data)
    date0 = datetime.datetime.strptime('1900-01-01', "%Y-%m-%d")

    lessdates = []
    basedatum = data.datum[:]
    for i in range(basedatum[0], basedatum[-1], 1000):
        lessdates.append(date0 + datetime.timedelta(days=int(i)))
    dates = lessdates
    
    press = data.press[:]
    print(press)
    nightadj = data[tarvar][0,:]
    dayadj = data[tarvar][1,:]    
    
    c = cdsapi.Client()
    r = c.retrieve('insitu-comprehensive-upper-air-observation-network',
                   {'variable': cdsvar,
                    'optional':['wind_bias_estimate'],
                    'statid': station,
                    'pressure_level':[10,20,30,50,70,100,150,200,250,300,400,500,700,850,925,1000]
                   }
                  )
    r.download(target='download.zip')
    assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
    z = zipfile.ZipFile('download.zip')
    z.extractall(path='./tocompare/wind/cds_'+station)
    z.close()
    cdsfile = glob.glob('./tocompare/wind/cds_'+ station +'/*.nc')
    cdsdata=eua.CDMDataset(cdsfile[0]).to_dataframe()
    daycdsdata = cdsdata[cdsdata.time.dt.hour > 6][cdsdata.time.dt.hour <= 18]
    nightcdsdata = cdsdata[cdsdata.time.dt.hour <= 6].append(cdsdata[cdsdata.time.dt.hour > 18])
    
    
    calc_day_out = {}
    calc_night_out = {}
    cds_day_out = {}
    cds_night_out = {}
    for i in [10,20,30,50,70,100,150,200,250,300,400,500,700,850,925,1000]:
        calc_day_out[str(i)]=[]
        calc_night_out[str(i)]=[]
        cds_day_out[str(i)]=[]
        cds_night_out[str(i)]=[]
        
    for i in range(len(dates)-1):
        for j in range(len(press)):
            try:
                d_adjd = dayadj[j][i*300]
                calc_day_out[str(int(press[j]))].append(d_adjd)
            except:
                calc_day_out[str(int(press[j]))].append(np.nan) 
            try:
                d_cdsd = daycdsdata[daycdsdata.plev == press[j]*100][daycdsdata.time >= dates[i]][daycdsdata.time < dates[i+1]]
                cds_day_out[str(int(press[j]))].append(d_cdsd[var].iloc[0])
            except:
                cds_day_out[str(int(press[j]))].append(np.nan)
            try:
                n_cdsd = nightcdsdata[nightcdsdata.plev == press[j]*100][nightcdsdata.time >= dates[i]][nightcdsdata.time < dates[i+1]]
                cds_night_out[str(int(press[j]))].append(n_cdsd[var].iloc[0])
            except:
                cds_night_out[str(int(press[j]))].append(np.nan)
            try:
                n_adjd = nightadj[j][i*300]
                calc_night_out[str(int(press[j]))].append(n_adjd)
            except:
                calc_night_out[str(int(press[j]))].append(np.nan)
                
    return dates, cds_day_out, cds_night_out, calc_day_out, calc_night_out

In [3]:
# def plt_adj(cdata, adata, plev, station, time, adj, save):
#     cdata = cdata[cdata.plev == plev*100]
#     adata = adata[adata.plev == plev*100]
#     fig, ax = plt.subplots()
#     ax.scatter(cdata['time'], cdata.wind_bias_estimate, label='CDS')
#     ax.scatter(adata['time'], adata.wind_bias_estimate, label='Calc')

#     ax.set(xlabel='time', ylabel='Adjustment: '+adj,
#            title= str(station)+' Temperature '+str(plev)+' hPa '+time)
#     ax.legend()
#     ax.grid()
    
#     if save:
#         fig.savefig(str(station)+'_Temperature_'+str(plev)+'_hPa_'+time+'.png')
#     plt.show()
#     return

In [4]:
def plt_adj(dates, cdata, adata, plev, station, time, adj, save):
#     dates_list = [datetime.datetime.strptime(date, '%Y%m%d').date() for date in dates[1:]]
    fig, ax = plt.subplots()
    ax.scatter(dates[1:], cdata[str(plev)], label='CDS')
    ax.scatter(dates[1:], adata[str(plev)], label='Calc')

    ax.set(xlabel='breaks', ylabel='Adjustment: '+adj,
           title= str(station)+' Temperature '+str(plev)+' hPa '+time)
    ax.legend()
    ax.grid()
    
    if save:
        fig.savefig(str(station)+'_Temperature_'+str(plev)+'_hPa_'+time+'.png')
    plt.show()
    return

In [ ]:
station = '70219'
adj = 'directionbias'
dates, dcd, ncd, dd, nd = get_adj(station, adj)

plt_adj(dates, dcd, dd, 100, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 100, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 500, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 500, station, 'night', adj, save=False)
plt_adj(dates, dcd, dd, 850, station, 'day', adj, save=False)
plt_adj(dates, ncd, nd, 850, station, 'night', adj, save=False)

File: <HDF5 file "feedbackmergedwinddir070219.nc" (mode r)>
Permissions: RW (a1400070)
Filesize: 20.94 MB
Filename: ./Wind_adjustment/070219/feedbackmergedwinddir070219.nc
(G)roups/(V)ariables: 

 - V | bias_estimate________________________________ : : (2, 16, 28125)
 - V | datum________________________________________ : : (28125,)
 - V | directionbias________________________________ : : (2, 16, 28125)
 - V | era5_fgdep___________________________________ : : (2, 16, 28125)
 - V | hour_________________________________________ : : (2,)
 - V | hours________________________________________ : : (2, 28125)
 - V | lat__________________________________________ : : (1,)
 - V | lon__________________________________________ : : (1,)
 - V | press________________________________________ : : (16,)
 - V | pressure_____________________________________ : : (16,)
 - V | station______________________________________ : : (1,)
 - V | time_________________________________________ : : (28125,)
 - V | uwindbi

2021-05-04 15:26:07,883 INFO Welcome to the CDS
2021-05-04 15:26:07,884 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network


[  10.   20.   30.   50.   70.  100.  150.  200.  250.  300.  400.  500.
  700.  850.  925. 1000.]


2021-05-04 15:26:08,177 INFO Request is completed
2021-05-04 15:26:08,178 INFO Downloading http://136.156.132.176/cache-compute-0000/cache/data2/adaptor.comprehensive_upper_air.retrieve-1620130746.408693-7818-3-dffbac9b-256c-4e83-85c5-09063c1bd783.zip to download.zip (21.9M)
2021-05-04 15:26:09,319 INFO Download rate 19.2M/s  
2021-05-04 15:26:09,400 INFO Reading Profile on slice(None, None, None)
2021-05-04 15:26:13,411 INFO Note: NumExpr detected 56 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [ ]:
station = '10393'
dates, dcd, ncd, dd, nd = get_adj(station, 'directionbias')

plt_adj(dates, cd, ad, 100, station, 'day', adj, save=False)
plt_adj(dates, cn, an, 100, station, 'night', adj, save=False)
plt_adj(dates, cd, ad, 500, station, 'day', adj, save=False)
plt_adj(dates, cn, an, 500, station, 'night', adj, save=False)
plt_adj(dates, cd, ad, 850, station, 'day', adj, save=False)
plt_adj(dates, cn, an, 850, station, 'night', adj, save=False)

In [ ]:
station = '11035'
dates, dcd, ncd, dd, nd = get_adj(station, 'directionbias')

plt_adj(dates, cd, ad, 100, station, 'day', adj, save=False)
plt_adj(dates, cn, an, 100, station, 'night', adj, save=False)
plt_adj(dates, cd, ad, 500, station, 'day', adj, save=False)
plt_adj(dates, cn, an, 500, station, 'night', adj, save=False)
plt_adj(dates, cd, ad, 850, station, 'day', adj, save=False)
plt_adj(dates, cn, an, 850, station, 'night', adj, save=False)

In [ ]:
station = '91413'
dates, dcd, ncd, dd, nd = get_adj(station, 'directionbias')

plt_adj(dates, cd, ad, 100, station, 'day', adj, save=False)
plt_adj(dates, cn, an, 100, station, 'night', adj, save=False)
plt_adj(dates, cd, ad, 500, station, 'day', adj, save=False)
plt_adj(dates, cn, an, 500, station, 'night', adj, save=False)
plt_adj(dates, cd, ad, 850, station, 'day', adj, save=False)
plt_adj(dates, cn, an, 850, station, 'night', adj, save=False)